In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import pandas as pd

import numpy as np

In [2]:
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    # Restrict TensorFlow to only allocate 4GB of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0],
                                                                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
import pickle

In [4]:
with open('./pkl_files/enchanced/validation/en_val_data.pkl', 'rb') as f:
    val_img_arr = pickle.load(f)
with open('./pkl_files/enchanced/validation/en_val_data_age.pkl', 'rb') as f:
    val_bone_age_arr = pickle.load(f)
with open('./pkl_files/enchanced/validation/en_val_data_gender.pkl', 'rb') as f:
    val_gender_arr = pickle.load(f)

images_arr_val = np.array(val_img_arr)
bone_age_val = np.array(val_bone_age_arr)
gender_val = np.array(val_gender_arr)

In [5]:
with open('./pkl_files/enchanced/training/en_train_data.pkl', 'rb') as f:
    train_img_arr = pickle.load(f)
with open('./pkl_files/enchanced/training/en_train_data_age.pkl', 'rb') as f:
    train_bone_age_arr = pickle.load(f)
with open('./pkl_files/enchanced/training/en_train_data_gender.pkl', 'rb') as f:
    train_gender_arr = pickle.load(f)

images_arr_train = np.array(train_img_arr)
bone_age_train = np.array(train_bone_age_arr)
gender_train = np.array(train_gender_arr)

In [6]:
X_train = images_arr_train
X_val = images_arr_val
y_train = bone_age_train
y_val = bone_age_val
print("Training set: (",len(X_train),", ",len(y_train),")\n")
print("Validation set: (",len(X_val),", ",len(y_val),")\n")

Training set: ( 12611 ,  12611 )

Validation set: ( 1425 ,  1425 )


In [7]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Concatenate
from keras.utils import plot_model

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, Input, Flatten, Dropout
from tensorflow.keras.layers import Activation, Concatenate, Conv2D, Multiply
from keras.applications.resnet import ResNet50
import keras
from keras.applications.inception_v3 import preprocess_input

In [18]:
def build_regression_model():
    base_model_carpal = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
    base_model_metacarpal = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
    gender_input = Input(shape=(1,), name='gender_input')
    
    for layer in base_model_carpal.layers:
        layer._name = 'carpal_' + layer.name
    for layer in base_model_metacarpal.layers:
        layer._name = 'metacarpal_' + layer.name
     
    for layer in base_model_carpal.layers:
        layer.trainable = False
    for layer in base_model_metacarpal.layers:
        layer.trainable = False
        
    x1 = Flatten()(base_model_carpal.output)
    x2 = Flatten()(base_model_metacarpal.output)

    gender_weight = tf.Variable(initial_value=1.0, trainable=True, name='gender_weight')
    gender_input_weighted = tf.multiply(gender_input, gender_weight)
    x = Concatenate()([x1, x2, gender_input_weighted])

    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    
    predictions = Dense(1, activation='relu')(x) 

    model = Model(inputs=(base_model_carpal.input, base_model_metacarpal.input, gender_input), outputs=predictions)

    return model

In [20]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 carpal_input_7 (InputLayer)    [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 metacarpal_input_8 (InputLayer  [(None, 224, 224, 3  0          []                               
 )                              )]                                                                
                                                                                                  
 carpal_conv1_pad (ZeroPadding2  (None, 230, 230, 3)  0          ['carpal_input_7[0][0]']         
 D)                                                                                         

In [19]:
model = build_regression_model()

In [37]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'mae'])

In [38]:
from keras.callbacks import ModelCheckpoint, EarlyStopping,CSVLogger

filepath = "saved_models/resnet/weights-improvement-{epoch:02d}-{accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor='accuracy',verbose=1,save_best_only=True,mode ='min')
early_stop = EarlyStopping(monitor='mae',patience=5,verbose=1)
log_csv = CSVLogger('./csv_files/resnet/test_train.csv',separator=',', append=False)

callbacks_list = [checkpoint, log_csv]

In [39]:
history = model.fit(
    X_train[:1000], y_train[:1000],
    epochs=10,
    batch_size=8,
    validation_data=(X_val[:100], y_val[:100]),
    callbacks=callbacks_list
)

Epoch 1/10



KeyboardInterrupt

